In [1]:
!pip install spacy
!pip install num2words

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
!python -m spacy download de_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 22.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [3]:
import os
import glob
import nemo.collections.asr as nemo_asr
import torch
import numpy as np
import ctc_decoders
import torchaudio
import soundfile as sf
import librosa
import pandas as pd
from num2words import num2words
import re

In [4]:
from processing.text_processor import preprocess_transcript_for_alignment, preprocess_transcript_for_sentence_split, split_to_sentences
from transcribing.stt_transcribe import get_stt_transcription, get_stt_probs

In [5]:
torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(torch.__version__)
print(device)

1.11.0
cuda


In [6]:
!nvidia-smi

Fri Jun 17 13:23:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:3B:00.0 Off |                    0 |
| N/A   43C    P0    69W / 350W |  12851MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Read true transcript
Read the text from the complete protocol provided by the parliament of canton Schwyz. After reading the text process everything and split the whole text into its sentences. 

In [7]:
PATH_TO_TRANSCRIPT = 'data/text_processed_2021-03-24.txt'

In [8]:
numeral_to_ordinal = {
    ' 1. ': ' erster ',
    ' 2. ': ' zweiter ',
    ' 3. ': ' dritter ',
    ' 4. ': ' vierter ',
    ' 5. ': ' fünfter ',
    ' 6. ': ' sechster ',
    ' 7. ': ' siebter ',
    ' 8. ': ' achter ',
    ' 9. ': ' neunter ',
    ' 10. ': ' zehnter ',
    ' 11. ': ' elfter ',
    ' 12. ': ' zwölfter ',
    ' 13. ': ' dreizehnter ',
    ' 14. ': ' vierzehnter ',
    ' 15. ': ' fünfzehnter ',
    ' 16. ': ' sechszehnter ',
    ' 17. ': ' siebzehnter ',
    ' 18. ': ' achtzehnter ',
    ' 19. ': ' neunzehnter ',
    ' 20. ': ' zwanzigster ',
    ' 21. ': ' einundzwanzigster ',
    ' 22. ': ' zweiundzwanzigster ',
    ' 23. ': ' dreiundzwanzigster ',
    ' 24. ': ' vierundzwanzigster ',
    ' 25. ': ' fünfundzwanzigster ',
    ' 26. ': ' sechsundzwanzigster ',
    ' 27. ': ' siebenundzwanzigster ',
    ' 28. ': ' achtundzwanzigster ',
    ' 29. ': ' neunundzwanzigster ',
    ' 30. ': ' dreissigster ',
    ' 31. ': ' einunddreissigster '
}

In [9]:
with open(PATH_TO_TRANSCRIPT, encoding='utf-8') as f:
    transcript = f.read()
    transcript = preprocess_transcript_for_sentence_split(transcript)
    for key in numeral_to_ordinal.keys():
        transcript = transcript.replace(key, numeral_to_ordinal[key])
    transcript = transcript.replace('%','Prozent')
    transcript = transcript.replace('Mio.','Millionen')
    transcript = transcript.replace('Fr.','Franken')
    transcript = transcript.replace('KRP','Kantonsratpräsident')
    transcript = transcript.replace('KR','Kantonsrat')
    transcript = transcript.replace('Dr.','Doktor')
    transcript = transcript.replace('§§','Paragraph')
    transcript = transcript.replace('§','Paragraph')
    transcript = transcript.replace('Abs.','Absatz')
    transcript = transcript.replace('z. B.','zum Beispiel')
    transcript = re.sub(r"(\d+)", lambda x: num2words(int(x.group(0)), lang='de'), transcript)
    truth_sentences = split_to_sentences(transcript)

In [10]:
#print(truth_sentences)

## Retrieve all audio files to later match with the corresponding text

For testing use a test file randomly extracted from one of the audio files

In [11]:
# set a single pregenerated file for testing
AUDIO_FILE = ['data/final_0324.wav']

## Transcription
Load and instantiate a pre-trained model from NVIDIA. In this case we use CTC decoding as output. 

In [12]:
# load model once
model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_de_quartznet15x5")

[NeMo I 2022-06-17 13:23:54 cloud:56] Found existing object /home/user/.cache/torch/NeMo/NeMo_1.9.0/stt_de_quartznet15x5/b82da4f9a238e099c0d94a6a0f7e8b4d/stt_de_quartznet15x5.nemo.
[NeMo I 2022-06-17 13:23:54 cloud:62] Re-using file from: /home/user/.cache/torch/NeMo/NeMo_1.9.0/stt_de_quartznet15x5/b82da4f9a238e099c0d94a6a0f7e8b4d/stt_de_quartznet15x5.nemo
[NeMo I 2022-06-17 13:23:54 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-06-17 13:23:55 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /raid/noneval.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ä
    - ö
    - ü
    - ß
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    num_workers: 8
    pin_memory: true
    
[NeMo W 2022-06-17 13:23:55 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    V

[NeMo I 2022-06-17 13:23:55 features:200] PADDING: 16


[NeMo W 2022-06-17 13:23:56 nemo_logging:349] /home/user/.local/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                    not been set for this class (WER). The property determines if `update` by
                    default needs access to the full metric state. If this is not the case, significant speedups can be
                    achieved and we recommend setting this to `False`.
                    We provide an checking function
                    `from torchmetrics.utilities import check_forward_no_full_state`
                    that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                    default for now) or if `full_state_update=False` can be used safely.
                    
      warnings.warn(*args, **kwargs)
    


[NeMo I 2022-06-17 13:23:57 save_restore_connector:243] Model EncDecCTCModel was successfully restored from /home/user/.cache/torch/NeMo/NeMo_1.9.0/stt_de_quartznet15x5/b82da4f9a238e099c0d94a6a0f7e8b4d/stt_de_quartznet15x5.nemo.


Retrieve the transcript of the given audio sequence

In [13]:
stt_output = get_stt_transcription(model, AUDIO_FILE)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
#print(stt_output)

For comparison we attach the actual true transcript of the given sequence

In [15]:
!pip install biopython

Defaulting to user installation because normal site-packages is not writeable


Get the probabilities for each token in the transcript

In [16]:
def get_stt_probs(model, file):
    logits = model.transcribe(paths2audio_files=file, logprobs=True)[0]
    probs = softmax(logits)
    return probs

def get_stt_logits(model, file):
    logits = model.transcribe(paths2audio_files=file, logprobs=True)[0]
    return logits

def softmax(logits):
    e = np.exp(logits - np.max(logits))
    return e / e.sum(axis=-1).reshape([logits.shape[0], 1])

In [17]:
stt_probs = get_stt_probs(model, AUDIO_FILE)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
stt_logits = get_stt_logits(model, AUDIO_FILE)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
#print(stt_logits)

## Timestep extraction
Define the duration of a timestep 

In [20]:
# 40ms is duration of a timestep at output of the model
time_stride = 0.02

Get the model's alphabet and replace unknown token with 'space'

In [21]:
labels = list(model.cfg.decoder.vocabulary) + ['blank']
labels[0] = 'space'

In [22]:
print(labels)

['space', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ä', 'ö', 'ü', 'ß', 'blank']


In [23]:
try:
    from plotly import graph_objects as go
except ModuleNotFoundError:
    !pip install plotly
    from plotly import graph_objects as go

In [24]:
'''
# plot probability distribution over characters for each timestep
fig_probs = go.Figure(
    go.Heatmap(z=stt_probs.transpose(),
               colorscale=[
                   [0, 'rgb(30,62,62)'],
                   [1, 'rgb(30,255,30)'],
               ],
               y=labels,
               dx=time_stride,
               name='Probs',
               hovertemplate='Time: %{x:.2f} s<br>Character: %{y}<br>Probability: %{z:.2f}<extra></extra>'),
    layout={
        'height': 300,
        'xaxis': {'title': 'Time, s'},
        'yaxis': {'title': 'Characters'},
        'title': 'Character Probabilities',
        'margin': dict(l=0, r=0, t=40, b=0, pad=0),
    }
)
fig_probs.show()
'''

"\n# plot probability distribution over characters for each timestep\nfig_probs = go.Figure(\n    go.Heatmap(z=stt_probs.transpose(),\n               colorscale=[\n                   [0, 'rgb(30,62,62)'],\n                   [1, 'rgb(30,255,30)'],\n               ],\n               y=labels,\n               dx=time_stride,\n               name='Probs',\n               hovertemplate='Time: %{x:.2f} s<br>Character: %{y}<br>Probability: %{z:.2f}<extra></extra>'),\n    layout={\n        'height': 300,\n        'xaxis': {'title': 'Time, s'},\n        'yaxis': {'title': 'Characters'},\n        'title': 'Character Probabilities',\n        'margin': dict(l=0, r=0, t=40, b=0, pad=0),\n    }\n)\nfig_probs.show()\n"

Extract timestamps for 'space' tokens

In [25]:
# get timestamps for space symbols
spaces = []

state = ''
idx_state = 0

if np.argmax(stt_probs[0]) == 0:
    state = 'space'

for idx in range(1, stt_probs.shape[0]):
    current_char_idx = np.argmax(stt_probs[idx])
    if state == 'space' and current_char_idx != 0 and current_char_idx != 31:
        spaces.append([idx_state, idx-1])
        state = ''
    if state == '':
        if current_char_idx == 0:
            state = 'space'
            idx_state = idx

if state == 'space':
    spaces.append([idx_state, len(stt_probs)-1])

In [26]:
no_conf = []
char_argmax = []
char_probs = []

conf_state = ''
index_state = 0

for index in range(1, stt_probs.shape[0]):
    current_char_index = np.argmax(stt_probs[index])
    if conf_state == 'space' and current_char_index != 0 and current_char_index != 31:
        no_conf.append([index_state, index-1])
        conf_state = ''
    if conf_state == '':
        if current_char_index != 31:
            max_index = np.argmax(stt_probs[index])
            char_argmax.append(max_index)
            char_probs.append(stt_probs[index][max_index])
        if current_char_index == 0:
            conf_state = 'space'
            index_state = index


In [27]:
vocab_dict = { i : labels[i] for i in range(0, len(labels), 1)}

In [28]:
print(vocab_dict)

{0: 'space', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 27: 'ä', 28: 'ö', 29: 'ü', 30: 'ß', 31: 'blank'}


In [29]:
char_tokens = list(map(vocab_dict.get, char_argmax))

In [30]:
#print(char_tokens)

In [31]:
average_probs = []
current_prob = []
for idx, token in enumerate(char_tokens):
    if token != 'space':
        current_prob.append(char_probs[idx])
    if token == 'space':
        if idx == 0:
            average_probs.append(1)
            continue
        average_probs.append(np.mean(current_prob))
        current_prob = []
        
    

In [32]:
print(len(average_probs))

19686


split audio signal into separate words

In [33]:
from IPython.display import Audio, display

In [34]:
#signal, sample_rate = librosa.load(TEST_FILE[0], sr=None)

In [35]:
#print(stt_output)

In [36]:
storage_words = list()
storage_start_time = list()
storage_end_time = list()

In [37]:
# calibration offset for timestamps: 180 ms
offset = -0.18

# split the transcript into words
words = stt_output.split()

# cut words
pos_prev = 0
for j, spot in enumerate(spaces):
    display(words[j])
    pos_end = offset + (spot[0]+spot[1])/2*time_stride
    #display(Audio(signal[int(pos_prev*sample_rate):int(pos_end*sample_rate)],
    #             rate=sample_rate))
    storage_words.append(words[j])
    storage_start_time.append(pos_prev)
    storage_end_time.append(pos_end)
    pos_prev = pos_end

#display(words[j+1])
#display(Audio(signal[int(pos_prev*sample_rate):],
#        rate=sample_rate))


'do'

'chasty'

'fra'

'lonedomencetcyparlon'

'dor'

...

'tronke'

'wünschee'

'gurtizit'

'lebesi'

'grundsätzi'

In [38]:
print(len(storage_start_time))

19686


In [39]:
processed_stt_output = pd.DataFrame(
    {'startTime': storage_start_time,
     'endTime': storage_end_time,
     'words': storage_words,
     'confidence': average_probs
    })

In [40]:
print(processed_stt_output)

       startTime   endTime                 words  confidence
0           0.00     20.69                    do    0.872931
1          20.69     21.75                chasty    0.596919
2          21.75     22.06                   fra    0.969434
3          22.06     31.93  lonedomencetcyparlon    0.819260
4          31.93     32.06                   dor    0.903856
...          ...       ...                   ...         ...
19681   12580.18  12580.70                tronke    0.924399
19682   12580.70  12581.54              wünschee    0.781795
19683   12581.54  12582.29              gurtizit    0.822112
19684   12582.29  12582.79                lebesi    0.875136
19685   12582.79  12583.40            grundsätzi    0.964152

[19686 rows x 4 columns]


## Alignment Procedure

In [41]:
from aligning.sentence_aligner import SentenceAligner
from aligning.stt_reader import NeMoSttOutputReader
from aligning.utils import Word

In [42]:
final_stt_output = []
for i in range(0, len(processed_stt_output)):
    final_stt_output.append(Word(storage_words[i], storage_start_time[i], storage_end_time[i], average_probs[i]))

print(len(final_stt_output))

19686


In [43]:
sentence_aligner = SentenceAligner()

In [44]:
sentence_alignment, alignment_info = sentence_aligner.predict_one(
    truth_sentences, final_stt_output,
    do_length_ratio_full_transcript_filtering=False,
    do_time_correction=False,
)

In [45]:
NONE_TIME_STRING_REPRESENTATION = '-1.0'
def transform_start_time(time):
    if time is None:
        return NONE_TIME_STRING_REPRESENTATION
    else:
        return time

def transform_end_time(time):
    if time is None:
        return NONE_TIME_STRING_REPRESENTATION
    else:
        return time

In [46]:
with open("output/final_output_0324.txt", 'w', encoding='utf-8') as f:
        for aligned_sentence in sentence_alignment:
            f.write(f'{transform_start_time(aligned_sentence.start_time)}\t{transform_end_time(aligned_sentence.end_time)}\t{aligned_sentence.sentence}\n')

## IOU Prediction
With the following features the IOU is predicted:
- Length Ratio
- Alignment Score
- Mean speech recognition confidence
- Chars per second

The estimator has been fitted in advanced based on a manually determined true transcript of the audio file.
Given the IOU a threshold is set determining whether the sentence is discarded for training or not. 

In [47]:
normalized_score = alignment_info.score / alignment_info.truth_length
length_ratio = alignment_info.truth_length / alignment_info.stt_length
chars_per_second = alignment_info.truth_length / alignment_info.audio_duration

features = pd.DataFrame(
                        {'confidence': alignment_info.stt_confidence,
                        'score': normalized_score,
                        'length_ratio': length_ratio,
                        'chars_per_second': chars_per_second
                        }
                       )
features['chars_per_second'].replace(np.inf, 0, inplace=True)
print(features)

      confidence     score  length_ratio  chars_per_second
0       0.865045 -0.624926      0.737705          1.342082
1       0.847847 -0.435248      1.470588         15.368852
2       0.914059 -0.354202      1.157895         16.500000
3       0.878132 -0.357314      1.021429         18.940397
4       0.898255 -0.358205      1.013216         14.705882
...          ...       ...           ...               ...
1571    0.929152 -0.471347      1.178082         15.221239
1572    0.901927 -0.454283      1.303249         16.094516
1573    0.906170 -0.423582      1.086957         19.230769
1574    0.903556 -0.443512      1.264151         23.633157
1575    0.873519 -0.385507      2.119048         27.639752

[1576 rows x 4 columns]


In [48]:
df_features = features.dropna()

In [49]:
print(len(df_features))

1576


In [50]:
df = df_features.copy()

In [51]:
print(df)

      confidence     score  length_ratio  chars_per_second
0       0.865045 -0.624926      0.737705          1.342082
1       0.847847 -0.435248      1.470588         15.368852
2       0.914059 -0.354202      1.157895         16.500000
3       0.878132 -0.357314      1.021429         18.940397
4       0.898255 -0.358205      1.013216         14.705882
...          ...       ...           ...               ...
1571    0.929152 -0.471347      1.178082         15.221239
1572    0.901927 -0.454283      1.303249         16.094516
1573    0.906170 -0.423582      1.086957         19.230769
1574    0.903556 -0.443512      1.264151         23.633157
1575    0.873519 -0.385507      2.119048         27.639752

[1576 rows x 4 columns]


Get estimated alignments of the audio file provided

## Estimate IOUs

In [52]:
!pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable


In [53]:
import pickle
from lightgbm import LGBMRegressor

In [54]:
estimator = pickle.load(open("iou_estimation/iou_estimator.pkl", "rb"))

In [55]:
ious = estimator.predict(df)

In [56]:
print(len(sentence_alignment))

1576


In [57]:
assert len(ious) == len(sentence_alignment)

In [58]:
print(np.mean(ious))

0.6380721604228268


## Filter dataset given IOU

In [59]:
IOU_THRESHOLD = 0.7

In [60]:
index_keep = [i for i in range(len(ious)) if ious[i] >= IOU_THRESHOLD]

In [61]:
high_quality_sentences = [sentence_alignment[i] for i in range(len(sentence_alignment)) if i in index_keep]

In [62]:
high_quality_sentences_info = alignment_info.loc[index_keep]

In [63]:
print(high_quality_sentences_info)

      truth_length  stt_length       score  stt_confidence  \
2              198         171  -70.132074        0.914059   
7               88          81  -22.760644        0.942351   
8              132         120  -53.719712        0.919147   
10              82          90  -44.867332        0.848072   
13              97          87  -43.986692        0.902157   
...            ...         ...         ...             ...   
1568            64          40  -27.474670        0.936327   
1569           121          94  -46.091020        0.933747   
1571            86          73  -40.535834        0.929152   
1572           361         277 -163.996281        0.901927   
1573           150         138  -63.537367        0.906170   

                                           truth_string  \
2     Als Erstes muss ich Ihnen leider mitteilen, da...   
7     Wie jedes Mal gebe ich kurz einige Information...   
8     Ich bitte Sie auch heute, die bis anhin vorbil...   
10    Jene, die sic

## Audio Splitting
Given the high quality sequences determined by the alignments with sentences, we split up the large audio files into digestable sequences. 
To prepare the sequences for training manifests for NeMo will be built. 

Prepare data for fitting the IOU estimator. First all the features are loaded and possible NAs are dropped from the dataframe. 
The actual IOUs are calculated given the start and end time of the truth sentence alignment and the predicted sentence alignment. 

In [64]:
from pydub import AudioSegment

audio_file = AudioSegment.from_wav(AUDIO_FILE[0])

for i, sentence in enumerate(high_quality_sentences):
    
    # file path for each sentence
    new_file_path = f'output/audio_output_0324/sentence_0324_{index_keep[i]}.wav'

    # splitting - works in milliseconds (*1000)

    start = sentence.start_time * 1000
    end = sentence.end_time * 1000
    
    new_audio_file = audio_file[start:end]

    # saving
    new_audio_file.export(new_file_path, format="wav")

In [65]:
high_quality_sentences_info.to_csv('output/audio_output_0324/sentence_info_0324.csv')